# 01 PUSH Type Import Strategy

## Use case
- Azure AI Search の サービス仕様ドキュメントをインプットにする。
  - https://learn.microsoft.com/ja-jp/azure/search/
- ドキュメントは OCR が必要。
- 開発者マニュアルは、構造化されたセクションとなっている。
- 各セクションは非常に詳細かつ専門性の高い技術解説が記載されており、ドキュメントサイズも大きい。
- ドキュメントには、テキスト、テーブル、図、グラフなどが含まれるが、ここでは、テキスト、テーブルデータのみを扱う。

## チャンキング設計
- Document Intelligence で、Markdown形式でテキストデータを抽出済み。
- 1つのドキュメントに大量のコンテキストが含まれており、ドキュメントサイズも大きいため、チャンキングを実施する。
- ドキュメントは、技術要素ごとに明確なセクションわけがされており、各セクションで見るとLLMが扱えないレベルのデータサイズではない。そのため、セクション単位でチャンキングする。
- 各チャンキングのContentはEmbeddingする。
- Overlapping は行わない。
- 広い意味のコンテキストを保持するために、上位2つのヘッダー（Markdown形式：#, ##）をメタデータとして保持する。例えば、検索インデックスに関する記載があった場合に、それが「キーワード検索」に属する情報なのか、「ベクトル検索」に属する情報なのかを判断するために保持する。

In [ ]:
! pip install python-dotenv langchain langchain-community langchain-openai langchainhub openai tiktoken azure-ai-documentintelligence azure-identity azure-search-documents==11.6.0b3 azure-ai-textanalytics

In [ ]:
import os

from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient
from azure.search.documents.indexes.models import (
    AIServicesVisionParameters,
    AIServicesVisionVectorizer,
    AIStudioModelCatalogName,
    AzureMachineLearningVectorizer,
    AzureOpenAIVectorizer,
    AzureOpenAIModelName,
    AzureOpenAIParameters,
    AzureOpenAIEmbeddingSkill,
    BlobIndexerDataToExtract,
    BlobIndexerParsingMode,
    CognitiveServicesAccountKey,
    DefaultCognitiveServicesAccount,
    ExhaustiveKnnAlgorithmConfiguration,
    ExhaustiveKnnParameters,
    FieldMapping,
    HnswAlgorithmConfiguration,
    HnswParameters,
    IndexerExecutionStatus,
    IndexingParameters,
    IndexingParametersConfiguration,
    InputFieldMappingEntry,
    KeyPhraseExtractionSkill,
    OutputFieldMappingEntry,
    ScalarQuantizationCompressionConfiguration,
    ScalarQuantizationParameters,
    SearchField,
    SearchFieldDataType,
    SearchIndex,
    SearchIndexer,
    SearchIndexerDataContainer,
    SearchIndexerDataIdentity,
    SearchIndexerDataSourceConnection,
    SearchIndexerIndexProjections,
    SearchIndexerIndexProjectionSelector,
    SearchIndexerIndexProjectionsParameters,
    SearchIndexerSkillset,
    SemanticConfiguration,
    SemanticField,
    SemanticPrioritizedFields,
    SemanticSearch,
    SimpleField,
    SplitSkill,
    VectorSearch,
    VectorSearchAlgorithmKind,
    VectorSearchAlgorithmMetric,
    VectorSearchProfile,
    VisionVectorizeSkill
)
from azure.search.documents.models import (
    HybridCountAndFacetMode,
    HybridSearch,
    SearchScoreThreshold,
    VectorizableTextQuery,
    VectorizableImageBinaryQuery,
    VectorizableImageUrlQuery,
    VectorSimilarityThreshold,
)
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv
from IPython.display import Image, display, HTML
from openai import AzureOpenAI
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

from langchain import hub
from langchain_openai import AzureChatOpenAI
from langchain_community.document_loaders import AzureAIDocumentIntelligenceLoader
from langchain_openai import AzureOpenAIEmbeddings
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.vectorstores.azuresearch import AzureSearch

In [ ]:
# Load environment variables
load_dotenv()

# Configuration
AZURE_AI_VISION_API_KEY = os.getenv("AZURE_AI_VISION_API_KEY")
AZURE_AI_VISION_ENDPOINT = os.getenv("AZURE_AI_VISION_ENDPOINT")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
BLOB_CONNECTION_STRING = os.getenv("BLOB_CONNECTION_STRING")
INDEX_NAME = "rag-search-index-push"
AZURE_SEARCH_ADMIN_KEY = os.getenv("AZURE_SEARCH_ADMIN_KEY")
AZURE_SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_ENDPOINT")
AZURE_AI_MULTI_SERVICE_ENDPOINT = os.getenv("AZURE_AI_MULTI_SERVICE_ENDPOINT")
AZURE_AI_MULTI_SERVICE_KEY = os.getenv("AZURE_AI_MULTI_SERVICE_KEY")
AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT")
AZURE_DOCUMENT_INTELLIGENCE_KEY = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_KEY")

### Load markdown file

In [ ]:
def read_file(filename):
    with open(filename, 'r', encoding="utf-8") as f:
        content = f.read()
    return content

### Splitting with Text Splitter

In [ ]:
from chunking.split_documents import split_markdown_headings

output_dir_documents = "../output/01_output/documents"
md_files = [f for f in os.listdir(output_dir_documents) if f.endswith('.md')]

splits_data = {}

for md_file in md_files:
    print("splits for {}".format(md_file))
    markdown_content = read_file(os.path.join(output_dir_documents, md_file))
    splits = split_markdown_headings(markdown_content)
    splits_data[md_file] = splits



# 03_Index-Design
Tips for Azure AI Search Index-Design

### インデックスの定義
- インデックスは単一で構成する。
- インプットドキュメントが説明的な内容のため、`Hybrid + Semantic Ranker` を採用する。そのため、ベクトル検索、 Semantic Ranker ための設定をする。

### フィールドの定義
- シンプルなパターンとして、ローカルのデータを検索インデックスにインポートさせる。
- ユーザのコンテキストを捉えるために、ベクトル検索を採用する。そのため、Embedding フィールドを構成する。
  - Embedding Model: `text-embedding-ada-002`を採用する。
- ドキュメントのヘッダー情報を`Metadata`フィールドに含める。
- ドキュメントのタイトルを`Title`フィールドに含める。
- ソースドキュメントの追跡用に `last_modified` を含める。

### Option: Enabling Semantic Ranker
Uses Microsoft’s language understanding models to rerank search results, enhancing relevance and providing results more aligned with the user’s context.

#### Implementation Considerations
Semantic Ranker を有効にする際の考慮事項を記載します。
- Semantic Ranker は、テキストクエリの BM25 でランク付けされた検索結果から、またはハイブリッド クエリの RRF でランク付けされた結果をリランキングします。
- 検索結果の数が 50 個を超える場合でも、リランキングが行われるのは上位 50 個の結果のみです。そのため、処理されない結果があることに注意してください。
- また、Semantic Ranker は、文章のコンテキストを理解させるために利用するため、適用対象のフィールドは説明的なものを指定することが推奨されます。ナレッジベース、オンラインドキュメントなど説明的なコンテンツを含むドキュメントでは、Semantic Ranker から最も多くのメリットが得られます。

In [ ]:
# User-specified parameter
USE_AAD_FOR_SEARCH = False  # Set this to False to use API key for authentication

def authenticate_azure_search(api_key=None, use_aad_for_search=False):
    if use_aad_for_search:
        print("Using AAD for authentication.")
        credential = DefaultAzureCredential()
    else:
        print("Using API keys for authentication.")
        if api_key is None:
            raise ValueError("API key must be provided if not using AAD for authentication.")
        credential = AzureKeyCredential(api_key)
    return credential

azure_search_credential = authenticate_azure_search(api_key=AZURE_SEARCH_ADMIN_KEY, use_aad_for_search=USE_AAD_FOR_SEARCH)

In [ ]:
# Creating Configuration for Semantic Ranker
def create_semantic_config():
	semantic_config = SemanticConfiguration(
		name="my-semantic-config",
		prioritized_fields=SemanticPrioritizedFields(
			title_field=SemanticField(field_name="title"),
			content_fields=[SemanticField(field_name="content")],
			keywords_fields=[SemanticField(field_name="key_phrases")],
		)
	)
	# Create the semantic settings with the configuration
	semantic_search = SemanticSearch(configurations=[semantic_config])
	return semantic_search

In [ ]:
def create_search_index(index_name, azure_openai_endpoint, azure_openai_embedding_deployment_id, azure_openai_key=None):
    return SearchIndex(
        name=index_name,
        fields=[
            SearchField(
                name="id",
                type=SearchFieldDataType.String,
                key=True,
                hidden=False,
                filterable=True,
                sortable=True,
                facetable=False,
                searchable=True,
                analyzer_name="keyword"
            ),
            SearchField(
                name="content",
                type=SearchFieldDataType.String,
                hidden=False,
                filterable=True,
                sortable=False,
                facetable=False,
                searchable=True,
                analyzer_name="ja.microsoft" # replace with your analyzer
            ),
            SearchField(
                name="title",
                type=SearchFieldDataType.String,
                hidden=False,
                filterable=True,
                sortable=False,
                facetable=False,
                searchable=True,
                analyzer_name="ja.microsoft" # replace with your analyzer
            ),
            SearchField(
                name="key_phrases",
                type=SearchFieldDataType.String,
                hidden=False,
                filterable=True,
                sortable=False,
                facetable=False,
                searchable=True,
                analyzer_name="ja.microsoft" # replace with your analyzer
            ),
            SearchField(
                name="vector",
                type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                hidden=False,
                filterable=False,
                sortable=False,
                facetable=False,
                searchable=True,
                vector_search_dimensions=1536,
                vector_search_profile_name="profile"
            ),
            SimpleField(
				name="last_modified",
                type=SearchFieldDataType.DateTimeOffset,
                hidden=False,
                filterable=True,
                sortable=True,
                facetable=False,
                searchable=False,
			),
        ],
        vector_search=VectorSearch(
			algorithms=[
				HnswAlgorithmConfiguration(
					name="myHnsw",
					parameters=HnswParameters(
						m=4,
						ef_construction=400,
						ef_search=500,
						metric=VectorSearchAlgorithmMetric.COSINE,
					),
				)
			],
			vectorizers=[
				AzureOpenAIVectorizer(
					name="myAzureOpenAIVectorizer",
					kind="azureOpenAI",
					azure_open_ai_parameters=AzureOpenAIParameters(
						resource_uri=azure_openai_endpoint,
						api_key=azure_openai_key,
						deployment_id=azure_openai_embedding_deployment_id,
						model_name=AzureOpenAIModelName.TEXT_EMBEDDING_ADA002,
					),
				)
			],
			profiles=[
				VectorSearchProfile(
					name="profile",
					algorithm_configuration_name="myHnsw",
					vectorizer="myAzureOpenAIVectorizer",
				)
			],
    	),
        semantic_search=create_semantic_config() # Here we add the semantic search configuration
	)

index = create_search_index(
    INDEX_NAME,
    AZURE_OPENAI_ENDPOINT,
    "text-embedding-ada-002", # replace with your deployment name
    AZURE_OPENAI_API_KEY
)
index_client = SearchIndexClient(
    endpoint=AZURE_SEARCH_ENDPOINT, credential=azure_search_credential
)
index_client.create_or_update_index(index)

print(f"Created index: {INDEX_NAME}")

#### Extracting Key Phrase: Using Azure AI Language
- We need to consider service limits for Azure AI Language: https://learn.microsoft.com/en-us/azure/ai-services/language-service/concepts/data-limits

In [ ]:
# Authenticate the client using your key and endpoint 
def authenticate_client():
    ta_credential = AzureKeyCredential(AZURE_AI_MULTI_SERVICE_KEY)
    text_analytics_client = TextAnalyticsClient(
            endpoint=AZURE_AI_MULTI_SERVICE_ENDPOINT, 
            credential=ta_credential,
            default_language="ja" # replace with your language
            )
    return text_analytics_client

def key_phrase_extraction_ai_language(client, documents):

    try:

        response = client.extract_key_phrases(documents = documents)[0]
        return response

    except Exception as err:
        print("Encountered exception. {}".format(err))


#### Extracting Key Phrage: Using Azure OpenAI

In [ ]:
def key_phrase_extraction_gpt(content):
    client = AzureOpenAI(
		azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
		api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
		api_version="2024-02-01"
	)
    system_message = f"""
    system:
	Extract the key phrases from the following text and return them as an array of strings: [ "Key phrase 1", "Key phrase 2", "Key phrase 3", ... ]. Ensure that each key phrase captures a significant concept or idea from the text.
	You can contain the maximum of 20 key phrases.
	
	# Few-shot Exapmle:
	Example 1:
	Text: "Artificial Intelligence is transforming industries by automating processes and enhancing decision-making through data analysis."
	Key Phrases: [ "Artificial Intelligence", "transforming industries", "automating processes", "enhancing decision-making", "data analysis" ]

	Example 2:
	Text: "Climate change is a pressing global issue that requires immediate action to reduce carbon emissions and protect ecosystems."
	Key Phrases: [ "Climate change", "pressing global issue", "immediate action", "carbon emissions", "ecosystems" ]
 
	User: 
	"""
    message_text = [
		{"role":"system","content": system_message},
		{"role":"user","content": content}
	]
    completion = client.chat.completions.create(
		model="gpt-4o", # model = "deployment_name"
		messages = message_text,
		# response_format={"type": "json_object"},
		temperature=0,
		)
    return completion.choices[0].message.content

#### Generate Embeddings

In [ ]:
# Generate Document Embeddings using OpenAI Ada 002
def generate_document_embeddings(content):
	client = AzureOpenAI(
		azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
		api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
		api_version="2024-02-01"
	)
	title_response = client.embeddings.create(input=content, model="text-embedding-ada-002")
	return title_response.data[0].embedding

#### Get update date


In [ ]:
import os
from datetime import datetime, timezone

def get_file_modification_time(file_path):
    # Get the modification time
    mod_time = os.path.getmtime(file_path)
    # Convert to a human-readable format
    mod_time_utc = datetime.fromtimestamp(mod_time, tz=timezone.utc)
    return mod_time_utc.isoformat()

### Insert text and embeddings into Azure AI Search Index

In [ ]:
import uuid

documnents_for_upload = []

for md_file, splits in splits_data.items():
	print("Processing {}".format(md_file))
	for split in splits:
		# print("Processing split {}".format(md_file))
		title = md_file
		content = split.page_content
		key_phrases = key_phrase_extraction_gpt(content)
		vector = generate_document_embeddings(content)
		last_modified = get_file_modification_time(os.path.join(output_dir_documents, md_file))
		documnents_for_upload.append(
			{
				"id": str(uuid.uuid4()),
				"content": content,
				"title": title,
				"key_phrases": key_phrases,
				"vector": vector,
				"last_modified": last_modified
			}
		)
  
documnents_for_upload

In [ ]:
search_client = SearchClient(endpoint=AZURE_SEARCH_ENDPOINT, index_name=INDEX_NAME, credential=azure_search_credential)
search_client.merge_or_upload_documents(documnents_for_upload)

# 04_Query-Design
Tips for Azure AI Search Query-Design

### Simple Search
- ユーザのクエリのコンテキストをとらえたい、かつサービスに特化したワードがクエリに含まれる可能性が高いため、Hybrid（フルテキスト検索＋ベクトル検索）＋Semantic Ranker を採用する。
  - Hybrid検索のスコアはAzure AI Searchでは、Reciprocal Rank Fusion (RRF) が採用される。 
- クエリはユーザのクエリをそのまま検索インデックスのクエリに利用する。

In [ ]:
search_client = SearchClient(endpoint=AZURE_SEARCH_ENDPOINT, index_name=INDEX_NAME, credential=azure_search_credential)

In [ ]:
query="ベクトル検索時の設定要素について教えてください"

vector_query = VectorizableTextQuery(
    text=query,
    k_nearest_neighbors=50,
    fields="vector",
)

# Perform the search
results = search_client.search(
    query_type='semantic',
    query_language='ja',
    semantic_configuration_name='my-semantic-config',
    search_text=query,
    vector_queries=[vector_query],
    top=5,
    select="content, title, key_phrases",
	search_fields=["content", "title", "key_phrases"],
)

for result in results:
    print(result)

### Query Expansion
- ユーザのクエリのコンテキストをとらえたい、かつサービスに特化したワードがクエリに含まれる可能性が高いため、Hybrid（フルテキスト検索＋ベクトル検索）＋Semantic Ranker を採用する。
  - Hybrid検索のスコアはAzure AI Searchでは、Reciprocal Rank Fusion (RRF) が採用される。 
- また、ユーザクエリから検索クエリを新しく生成する。
  - クエリはユーザのクエリをスタンドアローンなクエリに変換する。
  - また、検索のカバレッジを大きくするために、類似した入力クエリを複数生成する。

In [ ]:
import re
import os
from openai import AzureOpenAI
import json

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-02-01"
)

system_message = """
# Your Task
- Given the following conversation history and the users next question,rephrase the question to be a stand alone question.
- You also need to extend the original question to generate 5 related queries. This is done to capture the broader context of the user's question.
- You must output json format. In other words, You must output array of questions that length is 5.

# Json format example:
{
	"questions": [
		"related question 1",
		"related question 2",
		"related question 3",
		"related question 4",
		"related question 5"
	]
}
"""

def generate_expanded_query(text):
    message_text = [
		{"role":"system","content": system_message},
		{"role":"user","content": text}
	]
    completion = client.chat.completions.create(
		model="gpt-4o", # model = "deployment_name"
		messages = message_text,
		response_format={"type": "json_object"},
		temperature=0,
		)
    return completion.choices[0].message.content


In [ ]:
query="ベクトル検索時の設定要素について教えてください"
expanded_query = generate_expanded_query(query)
parsed_data = json.loads(expanded_query)
parsed_data

In [ ]:
for question in parsed_data["questions"]:
	vector_query = VectorizableTextQuery(
		text=question,
		k_nearest_neighbors=50,
		fields="vector",
	)
	# Perform the search
	results = search_client.search(
		query_type='semantic',
  		query_language='ja',
    	semantic_configuration_name='my-semantic-config',
		search_text=query,
		vector_queries=[vector_query],
		top=5,
		select="content, title, key_phrases",
		search_fields=["content", "title", "key_phrases"],
	)
	print("query: ", question)
	for result in results:
		print(result)
	print("\n")

### HyDE (Hypothetical Document Embeddings)
- ユーザのクエリのコンテキストをとらえたい、かつサービスに特化したワードがクエリに含まれる可能性が高いため、Hybrid（フルテキスト検索＋ベクトル検索）＋Semantic Ranker を採用する。
  - Hybrid検索のスコアはAzure AI Searchでは、Reciprocal Rank Fusion (RRF) が採用される。 
- また、ユーザクエリから検索クエリを新しく生成する。
  - クエリはユーザのクエリをスタンドアローンなクエリに変換する。
  - また、ユーザのクエリに基づいて仮想的な応答をLLMで作成し、それをベクトル変換した結果を用いて検索をかけるHyDEを採用します。クエリを検索対象のベクトルにより近いものに変換することで、検索精度を高めることを狙った手法です。
  - HyDEはLLMがまったく知識を持たないような領域だと役に立たない可能性があるため採用する際は注意してください。

In [ ]:
import re
import os
from openai import AzureOpenAI
import json

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-02-01"
)


def generate_hypothetical_query(text):
    hypothetical_gen_instruction = f"""Please write a passage to answer the question
	Question: {text}
	Passage:
	"""
    message_text = [
		{"role":"system","content": "You are an AI assistant."},
		{"role":"user","content": hypothetical_gen_instruction}
	]
    completion = client.chat.completions.create(
		model="gpt-4o", # model = "deployment_name"
		messages = message_text,
		# response_format={"type": "json_object"},
		temperature=0,
		)
    return completion.choices[0].message.content

In [ ]:
query="ベクトル検索時の設定要素について教えてください"
hypothetical_answer = generate_hypothetical_query(query)
hypothetical_answer

In [ ]:
vector_query = VectorizableTextQuery(
	text=hypothetical_answer,
	k_nearest_neighbors=50,
	fields="vector",
)
# Perform the search
results = search_client.search(
    query_type='semantic',
    query_language='ja',
    semantic_configuration_name='my-semantic-config',
	search_text=hypothetical_answer,
	vector_queries=[vector_query],
	top=5,
	select="content, title, key_phrases",
	search_fields=["content", "title", "key_phrases"],
)
for result in results:
	print(result)

## 05_Generate-Answer
検索インデックスから取得したものをコンテキストとして与えて、それをベースにした回答を生成させるプロンプトを設定します。
プロンプトエンジニアリングに関する包括的なガイダンスは以下を参照ください。

https://learn.microsoft.com/ja-jp/azure/ai-services/openai/concepts/prompt-engineering

In [ ]:
import re
import os
from openai import AzureOpenAI
import json

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-02-01"
)


def generate_answer(query, context):
    system_message = f"""
    system:
	You are an AI assistant that helps users answer questions given a specific context. You will be given a context and asked a question based on that context. Your answer should be as precise as possible and should only come from the context.
	Please add citation after each sentence when possible in a form "(Source: citation)". 
	context: {context}
	user: 
	"""
    message_text = [
		{"role":"system","content": system_message},
		{"role":"user","content": query}
	]
    completion = client.chat.completions.create(
		model="gpt-4o", # model = "deployment_name"
		messages = message_text,
		# response_format={"type": "json_object"},
		temperature=0,
		)
    return completion.choices[0].message.content

In [ ]:
context_text = ""
for result in results:
	context_text += result["content"] + " "

answer = generate_answer(query, context_text)
answer

## 06_Evaluation
RAG の Evaluation は、「検索評価」と「生成評価」にわけて実施することが推奨される。

### 検索評価 - 簡易評価
- シンプルな検索評価として、検索結果の上位5件にユーザクエリを解決するための情報が含まれているかどうかを評価します。
- クエリの種類は、想定されるエンドユーザーのクエリや、異なるドキュメントを答えとなるようなクエリを複数パターン用意します。
  - 回答に複数の文が必要な抽象的な質問：
  - 検索エンジンに一般的に入力されるものと同様の短縮されたクエリ：
  - 回答が質問とは異なる単語やフレーズを使用しているクエリ：
  - 回答が 1 つしかないクエリ
  - 複数の内容を質問しているクエリ
- ここではサンプルのため5パターンのクエリを用意しますが、包括的な評価をするためには、100件以上のパターンを用意することが推奨されます。（もちろん、ドキュメントの量やユーザのタスクによって異なるため、それぞれの要件にあわせて設計が必要です）
  - [参考：評価用データセットの作成](https://github.com/microsoft/promptflow-resource-hub/blob/main/sample_gallery/golden_dataset/copilot-golden-dataset-creation-guidance.md)


In [ ]:
# Test query set
queries = [
	"AI Search について勉強しています。ベクトル検索時の設定要素について教えてください",
	"ハイブリッド検索　メリット",
	"Azure AI Search には、リランクのモデルが利用できるか？",
	"フルテキスト検索の取得は最大何件か",
	"Hybrid検索とセマンティックランカーの特徴と違いはなんですか？"
]

#### シンプルクエリ

クエリを実行

In [ ]:
import pprint

# Test for simple query
for query in queries:
    vector_query = VectorizableTextQuery(
        text=query,
        k_nearest_neighbors=50,
        fields="vector",
    )

    # Perform the search
    results = search_client.search(
        query_type='semantic',
        query_language='ja',
    	semantic_configuration_name='my-semantic-config',
        search_text=query,
        vector_queries=[vector_query],
        top=5,
        select="content, title, key_phrases",
        search_fields=["content", "title", "key_phrases"],
    )

    print("Query:")
    pprint.pprint(query)
    print("\nResults:")
    for result in results:
        pprint.pprint(result)
        print("\n")

##### 1. 回答に複数の文が必要な抽象的な質問  
**質問:**  
*AI Search について勉強しています。ベクトル検索時の設定要素について教えてください*

- **評価:** 部分的に合致
  - **意図した回答が含まれるチャンクの順位:** 3位
  - **理由:** クエリに対応するベクトル検索の設定要素が具体的に記載されており、質問に直接対応していました。

##### 2. 検索エンジンに一般的に入力されるものと同様の短縮されたクエリ  
**質問:**  
*ハイブリッド検索 メリット*

- **評価:** 合致
  - **意図した回答が含まれるチャンクの順位:** 1位
  - **理由:** ハイブリッド検索のメリットや具体的な使用例、関連性の向上についての記述が見られ、クエリに対して的確に回答しています。

##### 3. 回答が質問とは異なる単語やフレーズを使用しているクエリ  
**質問:**  
*Azure AI Search には、リランクのモデルが利用できるか？*

- **評価:** 合致
  - **意図した回答が含まれるチャンクの順位:** 2位
  - **理由:** セマンティックランク付けについての詳細な説明があり、リランクに関する情報が提供されていますが、さらに具体的な言及が期待されます。

##### 4. 回答が 1 つしかないクエリ  
**質問:**  
*フルテキスト検索の取得は最大何件か*

- **評価:** 合致
  - **意図した回答が含まれるチャンクの順位:** 1位
  - **理由:** フルテキスト検索の最大取得件数に関する直接的な情報が提供され、クエリに対して明確に回答しています。

##### 5. 複数の内容を質問しているクエリ  
**質問:**  
*Hybrid検索とセマンティックランカーの特徴と違いはなんですか？*

- **評価:** 合致
  - **意図した回答が含まれるチャンクの順位:** 1位、2位
  - **理由:** ハイブリッド検索とセマンティックランカーの機能に関する比較が含まれており、クエリに対して適切な情報が提供されています。


#### HyDE

クエリを実行

In [ ]:
import pprint

# Test for simple query
for query in queries:
    hypothetical_answer = generate_hypothetical_query(query)   
    vector_query = VectorizableTextQuery(
		text=hypothetical_answer,
		k_nearest_neighbors=50,
		fields="vector",
	)

    # Perform the search
    results = search_client.search(
        query_type='semantic',
        query_language='ja',
    	semantic_configuration_name='my-semantic-config',
        search_text=hypothetical_answer,
        vector_queries=[vector_query],
        top=5,
        select="content, title, key_phrases",
        search_fields=["content", "title", "key_phrases"],
    )

    print("Query:")
    pprint.pprint(query)
    print("\nHyDE Query:")
    pprint.pprint(hypothetical_answer)
    print("\nResults:")
    for result in results:
        pprint.pprint(result)
        print("\n")

##### 1. 回答に複数の文が必要な抽象的な質問  
**質問:**  
*AI Search について勉強しています。ベクトル検索時の設定要素について教えてください*

- **評価:** 合致
  - **意図した回答が含まれるチャンクの順位:** 1位
  - **理由:** ベクトル検索に関連する設定要素（次元数、距離測定方法、インデックス構築など）が説明されています。

##### 2. 検索エンジンに一般的に入力されるものと同様の短縮されたクエリ  
**質問:**  
*ハイブリッド検索 メリット*

- **評価:** 合致
  - **意図した回答が含まれるチャンクの順位:** 2位
  - **理由:** ハイブリッド検索のメリット（精度の向上、セマンティックランク付けの効果、ベクトル検索とキーワード検索の統合など）が言及されています。

##### 3. 回答が質問とは異なる単語やフレーズを使用しているクエリ  
**質問:**  
*Azure AI Search には、リランクのモデルが利用できるか？*

- **評価:** 合致
  - **意図した回答が含まれるチャンクの順位:** 1位
  - **理由:** Azure AI Searchでセマンティックランク付けや再ランク付けのプロセスに関する情報が提供されています。

##### 4. 回答が 1 つしかないクエリ  
**質問:**  
*フルテキスト検索の取得は最大何件か*

- **評価:** 合致
  - **意図した回答が含まれるチャンクの順位:** 2位
  - **理由:** フルテキスト検索結果の取得件数に関する言及（既定の50件、最大1000件まで）が含まれています。

##### 5. 複数の内容を質問しているクエリ  
**質問:**  
*Hybrid検索とセマンティックランカーの特徴と違いはなんですか？*

- **評価:** 合致
  - **意図した回答が含まれるチャンクの順位:** 1位, 2位, 3位
  - **理由:** ハイブリッド検索とセマンティックランク付けに関するそれぞれの特徴についての言及が見られます。

### 検索評価 - 厳密な評価
TBD
- 評価指標の設計
- クエリパターン
- 評価用データの選定・設計

### 生成評価 - 簡易評価
- シンプルな検索評価として、検索結果の上位5件にユーザクエリを解決するための情報が含まれているかどうかを評価します。
- クエリの種類は、想定されるエンドユーザーのクエリや、異なるドキュメントを答えとなるようなクエリを複数パターン用意します。
  - 回答に複数の文が必要な抽象的な質問：
  - 検索エンジンに一般的に入力されるものと同様の短縮されたクエリ：
  - 回答が質問とは異なる単語やフレーズを使用しているクエリ：
  - 回答が 1 つしかないクエリ
  - 複数の内容を質問しているクエリ
- ここではサンプルのため5パターンのクエリを用意しますが、包括的な評価をするためには、100件以上のパターンを用意することが推奨されます。（もちろん、ドキュメントの量やユーザのタスクによって異なるため、それぞれの要件にあわせて設計が必要です）
  - [参考：評価用データセットの作成](https://github.com/microsoft/promptflow-resource-hub/blob/main/sample_gallery/golden_dataset/copilot-golden-dataset-creation-guidance.md)


In [ ]:
# Define RAG Pipeline
def rag_pipeline_with_hyde(query):
	hypothetical_answer = generate_hypothetical_query(query)
	vector_query = VectorizableTextQuery(
		text=hypothetical_answer,
		k_nearest_neighbors=50,
		fields="vector",
	)
	# Perform the search
	results = search_client.search(
		query_type='semantic',
		query_language='ja',
    	semantic_configuration_name='my-semantic-config',
		search_text=hypothetical_answer,
		vector_queries=[vector_query],
		top=5,
		select="content, title, key_phrases",
		search_fields=["content", "title", "key_phrases"],
	)
	
	context_text = ""
	for result in results:
		context_text += result["content"] + " "

	return generate_answer(query, context_text)

In [ ]:
# Test query set
queries = [
	"AI Search について勉強しています。ベクトル検索時の設定要素について教えてください",
	"ハイブリッド検索　メリット",
	"Azure AI Search には、リランクのモデルが利用できるか？",
	"フルテキスト検索の取得は最大何件か",
	"Hybrid検索とセマンティックランカーの特徴と違いはなんですか？"
]

In [ ]:
import time

for query in queries:
    start_time = time.time()
    answer = rag_pipeline_with_hyde(query)
    end_time = time.time()
    elapsed_time = end_time - start_time

    print(f"Query: {query}")
    print(f"Answer: {answer}")
    print(f"Elapsed time: {elapsed_time} seconds")
    print("\n")